In [3]:
import great_expectations as gx
import simple_salesforce
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)


sf_client = Salesforce(
    username=os.getenv("SF_USERNAME"),
    password=os.getenv("SF_PASSWORD"),
    security_token=os.getenv("SF_TOKEN")
)

def get_context_from_azure() -> yaml context:

context = gx.data_context.FileDataContext.create("/home/jovyan/gx_suites/poc")

    - No action was taken.
  warnings.warn(message)

    - No action was taken.
  warnings.warn(message)

INFO:great_expectations.data_context.data_context.file_data_context:FileDataContext loading fluent config
INFO:great_expectations.datasource.fluent.config:Loading 'datasources' ->
[]


In [4]:
ASSET_NAME = 'Property__c'

In [5]:
class GXContextFactory(object):

    def __init__(self, sf_client, gx_context, object_name):
        self.sf_client = sf_client
        self.gx_context = gx_context
        
        try:
            self.gx_suite = self.gx_context.add_expectation_suite(expectation_suite_name=object_name)
        except DataContextError:
            self.gx_suite = self.gx_context.get_expectation_suite(expectation_suite_name=object_name)
        
        object_metadata = getattr(sf_client, object_name).describe()
        
        for field_metadata in object_metadata['fields']:
            self._generate_expectations_for_field(field_metadata)

        self.gx_context.add_or_update_expectation_suite(expectation_suite=self.gx_suite)

    def _generate_expectations_for_field(self, field_metadata):
        if not field_metadata['nillable']:
            self._generate_non_null_expectation(field_metadata['name'])

        if field_metadata['unique']:
            self._generate_field_uniqueness_expectation(field_metadata['name'])
            
        if field_metadata['type'].lower() == 'picklist':
            value_set = [
                x['value'] 
                for x in field_metadata['picklistValues']
                if x['active']
            ]
            
            self._generate_picklist_expectations(
                field_metadata['name'],
                value_set
            )

    def _generate_picklist_expectations(self, field_name, value_set):
        expectation_configuration = ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_in_set",
            kwargs={
                "column": field_name,
                "value_set": value_set
            },
            meta={
                "notes": {
                    "format": "markdown",
                    "content": "Some clever comment about this expectation. **Markdown** `Supported`",
                }
            },
        )

        self.gx_suite.add_expectation(expectation_configuration=expectation_configuration)

    def _generate_non_null_expectation(self, field_name):
        expectation_configuration = ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={
                "column": field_name,
            },
            meta={
                "notes": {
                    "format": "markdown",
                    "content": "Some clever comment about this expectation. **Markdown** `Supported`",
                }
            },
        )

        self.gx_suite.add_expectation(expectation_configuration=expectation_configuration)

    def _generate_field_uniqueness_expectation(self, field_name):
        expectation_configuration = ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_unique",
            kwargs={
                "column": field_name,
            },
            meta={
                "notes": {
                    "format": "markdown",
                    "content": "Some clever comment about this expectation. **Markdown** `Supported`",
                }
            },
        )

        self.gx_suite.add_expectation(expectation_configuration=expectation_configuration)      

In [6]:
factory = GXContextFactory(sf_client, gx_context, 'Property__c')